## SQL miniproject Part 2

In [2]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

### creating local database engine for countryclub sqlite database

In [3]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [4]:
engine.table_names()
    

['Bookings', 'Facilities', 'Members']

##### To answer most of these questions I'm going to write a function that takes in the SQL query as a string and outputs a pandas dataframe.

In [5]:
def SQLquery(query_string):
    
    # open context manager
    with engine.connect() as con:
        rs = con.execute(query_string)
        df = pd.DataFrame(rs.fetchall())
        df.columns = rs.keys()
        
    return df

##### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [20]:
Q10Answer = SQLquery("""SELECT Facilities.name as Facility, 
SUM(CASE WHEN Bookings.memid = 0 THEN Bookings.slots*Facilities.guestcost
ELSE Bookings.slots*Facilities.membercost END) AS Revenue
FROM Bookings
INNER JOIN Facilities
ON Bookings.facid = Facilities.facid
GROUP BY Facilities.name
HAVING Revenue < 1000""" )

In [47]:
print(Q10Answer)

        Facility  Revenue
0     Pool Table      270
1  Snooker Table      240
2   Table Tennis      180


##### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [45]:
Q11Answer = SQLquery('''SELECT m.surname || ', ' || m.firstname AS Member_Name, 
CASE WHEN m.recommendedby IS NOT NULL THEN 
(SELECT surname || ', ' || firstname 
FROM Members AS mem 
WHERE m.recommendedby = mem.memid) 
END AS Recommender_Name
FROM Members as m ORDER BY Member_name ''')

In [46]:
print(Q11Answer)

                 Member_Name    Recommender_Name
0            Bader, Florence    Stibbons, Ponder
1                Baker, Anne    Stibbons, Ponder
2             Baker, Timothy     Farrell, Jemima
3                Boothe, Tim         Rownam, Tim
4            Butters, Gerald       Smith, Darren
5               Coplin, Joan      Baker, Timothy
6             Crumpet, Erica        Smith, Tracy
7                Dare, Nancy    Joplette, Janice
8             Farrell, David                None
9            Farrell, Jemima                None
10              GUEST, GUEST                None
11          Genting, Matthew     Butters, Gerald
12                Hunt, John  Purview, Millicent
13              Jones, David    Joplette, Janice
14            Jones, Douglas        Jones, David
15          Joplette, Janice       Smith, Darren
16           Mackenzie, Anna       Smith, Darren
17             Owen, Charles       Smith, Darren
18             Pinker, David     Farrell, Jemima
19        Purview, M

####  Q12: Find the facilities with their usage by member, but not guests 

In [77]:
Q12Answer = SQLquery('''SELECT Facilities.name AS facility_name, 
Bookings.memid, firstname || ' ' || surname as Member_Name, COUNT(Bookings.memid) as usage
FROM Facilities
INNER JOIN Bookings
ON Facilities.facid = Bookings.facid
INNER JOIN Members
ON Members.memid = Bookings.memid
GROUP BY Facilities.name, Bookings.memid
HAVING Bookings.memid <> 0''')

In [79]:
print(Q12Answer[['facility_name', 'Member_Name', 'usage'] ])

       facility_name        Member_Name  usage
0    Badminton Court       Darren Smith    132
1    Badminton Court        Tracy Smith     32
2    Badminton Court         Tim Rownam      4
3    Badminton Court     Gerald Butters     20
4    Badminton Court       Burton Tracy      2
..               ...                ...    ...
197   Tennis Court 2   Ramnaresh Sarwin     11
198   Tennis Court 2   Henrietta Rumney      1
199   Tennis Court 2      David Farrell      1
200   Tennis Court 2  Millicent Purview      1
201   Tennis Court 2          John Hunt      4

[202 rows x 3 columns]


####  Q13: Find the facilities usage by month, but not guests 

In [129]:
 Q13answer = SQLquery(''' SELECT Facilities.name as facility_name, month, count(month) as monthly_usage 
 FROM Facilities 
 INNER JOIN (SELECT memid, facid, strftime('%m',starttime) as month FROM Bookings) as subq
 ON Facilities.facid = subq.facid
 GROUP BY facility_name, month
 HAVING memid <> 0
 ''')

In [130]:
print(Q13answer)

      facility_name month  monthly_usage
0   Badminton Court    07             56
1   Badminton Court    08            146
2   Badminton Court    09            181
3    Massage Room 1    07            123
4    Massage Room 1    08            224
5    Massage Room 1    09            282
6    Massage Room 2    08             40
7        Pool Table    07            110
8        Pool Table    08            291
9        Pool Table    09            435
10    Snooker Table    07             75
11    Snooker Table    08            159
12    Snooker Table    09            210
13     Table Tennis    07             51
14     Table Tennis    08            147
15     Table Tennis    09            205
16   Tennis Court 1    07             88
17   Tennis Court 1    08            146
18   Tennis Court 2    08            149
